In [9]:
import os.path
import json
import datetime

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.auth.transport.requests import Request

In [12]:
SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]

def get_credentials():
    """Retrieve stored credentials from tokens.json."""
    if os.path.exists("tokens.json"):
        with open("tokens.json", "r") as token_file:
            tokens = json.load(token_file)
            return [Credentials.from_authorized_user_info(json.loads(token), SCOPES) for token in tokens]
    return []

def save_credentials(creds):
    """Save credentials to tokens.json."""
    tokens = []
    if os.path.exists("tokens.json"):
        with open("tokens.json", "r") as token_file:
            tokens = json.load(token_file)
    tokens.append(creds.to_json())
    with open("tokens.json", "w") as token_file:
        json.dump(tokens, token_file)

def main():
    """Shows basic usage of the Google Calendar API."""
    addNewAccount = False
    creds_list = get_credentials()
    if not creds_list:
        flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
        creds = flow.run_local_server(port=0)
        save_credentials(creds)
        creds_list = [creds]
    else if addNewAccount:
        flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
        creds = flow.run_local_server(port=0, authorization_prompt_message="Select the account to use or add a new one: ")
        save_credentials(creds)
        creds_list.append(creds)

    start_date = datetime.datetime(2023, 3, 15, 0, 0, 0).isoformat() + "Z"
    end_date = datetime.datetime(2024, 3, 16, 23, 59, 59).isoformat() + "Z"

    try:
        for creds in creds_list:
            service = build("calendar", "v3", credentials=creds)

            events_result = (
                service.events()
                .list(
                    calendarId="primary",
                    timeMin=start_date,
                    timeMax=end_date,
                    singleEvents=True,
                    orderBy="startTime",
                )
                .execute()
            )
            events = events_result.get("items", [])

            print("Events:")
            if not events:
                print("No events found.")
            else:
                for event in events:
                    start = event["start"].get("dateTime", event["start"].get("date"))
                    print(start, event["summary"])

    except Exception as error:
        print(f"An error occurred: {error}")
if __name__ == "__main__":
    main()


SyntaxError: expected ':' (2229899002.py, line 30)